In [153]:
import pandas as pd
import numpy as np
import scipy.sparse as sp
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_distances
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy.sparse.linalg import svds
from sklearn.metrics import roc_auc_score

pd.options.display.max_colwidth = 2000
pd.options.display.float_format = '{:.2f}'.format

Для решения задачи будем использовать те же данные, которые были использованы в скринкастах:

MovieLens — источник данных.

Предобработанные для обучения данные: история проставления оценок фильмам — ratings_df_sample_2.csv.

Предположим, постановка рейтинга — обязательное по итогам просмотра фильмов действие. Основываясь на этом, сгенерируйте новый целевой признак «факт просмотра фильма пользователем», который будет равен 1 для всех пар пользователь * фильм из подгруженного датасета.

А откуда взять «нолики»? В наших данных есть только пары пользователь * фильм, в которых пользователь точно смотрел фильм. Но для обучения модели нужны так называемые «негативы», то есть, пары, где пользователь фильм не смотрел. На практике приходится сталкиваться с необходимостью генерировать их вручную, давайте потренируемся это делать. 

Сначала найдите уникальные id всех пользователей и уникальные id всех фильмов.

С помощью функции random.choice (документация) сгенерируйте случайные пары пользователь * фильм

Поскольку среди сгенерированных пар могут быть и такие, что пользователь в них уже смотрел фильм, сгенерируйте побольше пар, например, удвоенное количество строк из источника. Это может занять пару минут.

Среди сгенерированных пар могут быть и дубликаты, удалите их.

Оставьте среди сгенерированных пар только те, в которых пользователь фильм не смотрел.

Возможно, пар получилось больше, чем нужно, выберите из них столько, сколько у нас строк в исходных данных.

Добавьте очищенные сгенерированные пары к исходным данным. Значение целевого признака в них будет равно нулю. Убедитесь, что у вас не появились дубликаты в датасете.

Подготовьте датасет к обучению: отделите тестовую часть от тренировочной.

Обучите dummy-model. Пусть она будет возвращать случайную вероятность принадлежности классу 1. Для этого можете использовать функцию random.random (документация). Оцените ее качество какой-то метрикой на свой вкус. Необходимо прогнозировать именно вероятность, чтобы была возможность ранжировать по ней варианты для рекомендации лучшего контента пользователю.

Реализуйте три алгоритма коллаборативной фильтрации: user-, item-based и алгоритм на основе матричной факторизации. Оцените их качество и адекватность. Если качество недостаточно хорошее, попробуйте варьировать параметры: количество похожих пользователей/фильмов, количество элементов в матрицах при матричном разложении.

Опишите вывод, содержащий информацию о том, какой алгоритм проявил себя лучше всего.

Оформите итоговое решение в Jupyter notebook.

Предобработка данных занимала много времени, поэтому я выполнил ее в отдельном ноутбуке
https://colab.research.google.com/drive/1UQlA8PIgyNbQGThU1dsxpBwo1arqO5mU?usp=sharing

In [157]:
!gdown 1nPNY-p806LIL68kTok2hmkcQ8taTs9Bg

Downloading...
From: https://drive.google.com/uc?id=1nPNY-p806LIL68kTok2hmkcQ8taTs9Bg
To: /content/data.csv
100% 618M/618M [00:09<00:00, 68.2MB/s]


In [168]:
df = pd.read_csv('data.csv')

<ipython-input-168-6456521db808>:1: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data.csv')


In [169]:
df.describe()

,userId,movieId,rating,timestamp,movie_viewed
count,12080198.00,12080198.00,12080198.00,12080198.00,12080198.00
mean,9748.79,477.77,1.78,557887167.49,0.50
std,5728.84,289.40,1.91,566096123.13,0.50
min,0.00,0.00,0.00,0.00,0.00
25%,4799.00,224.00,0.00,0.00,0.00
50%,9593.00,472.00,0.25,412417705.00,0.50
75%,14670.00,724.00,4.00,1111706236.00,1.00
max,19999.00,999.00,5.00,1427780469.00,1.00


Такое количество данных не вмещается в оперативную память гугл коллаба, пришлось уменьшить

In [160]:
# df = df.sample(frac=0.001, random_state=42)

In [161]:
# df = df.reset_index(drop=True)

In [170]:
df.head()

,userId,movieId,rating,timestamp,title,genres,movie_viewed
0,0,0,3.00,974918176.00,Jumanji (1995),Adventure|Children|Fantasy,1.00
1,1,0,3.50,1112061358.00,Jumanji (1995),Adventure|Children|Fantasy,1.00
2,2,0,2.00,1132728068.00,Jumanji (1995),Adventure|Children|Fantasy,1.00
3,3,0,2.00,1134476330.00,Jumanji (1995),Adventure|Children|Fantasy,1.00
4,4,0,3.00,1283448701.00,Jumanji (1995),Adventure|Children|Fantasy,1.00


In [171]:
train_data, test_data = train_test_split(df, test_size=0.01)

print('Train shape: {}'.format(train_data.shape))
print('Test shape: {}'.format(test_data.shape))

Train shape: (11959396, 7)
Test shape: (120802, 7)


In [172]:
n_users = train_data['userId'].nunique()
n_users

20000

In [173]:
n_movies = train_data['movieId'].nunique()
n_movies

1000

In [ ]:
%%time
train_data_matrix = np.zeros((n_users, n_movies))
for line in train_data.to_dict(orient='records'):
    try:
      train_data_matrix[line['userId'], line['movieId']] = int(line['movie_viewed'])
    except:
      print(line['userId'])
      print(line['movieId'])
      print(train_data_matrix[line['userId'], line['movieId']])
      print(line['movie_viewed'])
      break

In [142]:
def dummy_model(features):
  pred = []
  for i in features.values:
    pred.append(random.random())
  return pred

In [90]:
roc_auc_dummy_model = roc_auc_score(test_target,dummy_model(test_features))

In [91]:
roc_auc_dummy_model

0.569078947368421

# Алгоритм user-based

In [92]:
n_users = train['userId'].nunique()
n_movies = train['movieId'].nunique()

In [93]:
n_users

10029

In [94]:
n_movies

1720

In [95]:
train_matrix = np.array(pd.pivot_table(train,values='movie_viewed',index='userId',columns='movieId',fill_value=0))

In [96]:
train_matrix

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [97]:
user_similarity = cosine_distances(train_matrix)

In [108]:
movie_similarity = cosine_distances(train_matrix.T)

In [119]:
%%time
top=10
top_similar_users = []
for i in range(n_users):
    neighbors = (user_similarity[i]).argsort()[1:top + 1]
    top_similar_users.append(
        train_matrix[neighbors]
    )
top_similar_users = np.array(top_similar_users)


CPU times: user 3.57 s, sys: 906 ms, total: 4.48 s
Wall time: 5.43 s
